# Laboratorio 3(Regresion logistica One vs All) Grupo 1
 <h3>Implementacion del modelo de regresion logistica one-vs-all para realizar clasificacion de clases.<h3>
 <HR>
 <h3>
  NOMBRE: POLO ORELLANA BRAYAN SIMON <br>
  CARRERA: INGENIERIA DE SISTEMAS <BR>
  FECHA: 19/03/2024 <BR>

  * [Enlace de invitacion para ser colaborador](https://github.com/bspoloo/SIS420-012024/invitations)
  
  * [Enlace al git hub](https://github.com/bspoloo/SIS420-012024/tree/main/Laboratorios/Laboratorio%203)
  
  * [Enlace al Colab](https://colab.research.google.com/github/bspoloo/SIS420-012024/blob/main/Laboratorios/Laboratorio%203/Laboratorio%203.ipynb?hl=es)
  
 <h3>

Para el laboratorio se hizo uso del para entrenar el modelo de **Regresion Logistica one vs all** y dada la información relacionada con el crédito de una persona, se creo un modelo de aprendizaje automático que pueda clasificar el puntaje crediticio de una persona

 A lo largo de los años, se han recopilado datos bancarios básicos y mucha información relacionada con el crédito de varias personas. Se quiere construir un sistema inteligente para segregar a las personas en tramos de puntuación crediticia para reducir los esfuerzos manuales. Haciendo uso del siguiente dataset: [credit_score_classification_processed](https://www.kaggle.com/datasets/deepaksaipendyala/credit-score-classification-processed).

El archivo `credit_score.csv` contiene un conjunto de datos de entrenamiento para clasificar el puntaje crediticio de una persona.

## 1. Regresion Logistica One vs All

Importamos todas las librerias necesarias para el manejo del dataset.

In [526]:
# utilizando la libreria os para manejos de directorios
import os

# Computacion vectorial y cientifica para python
import numpy as np  

#importamos pandas para el manejo del dataset, y separarlos dentro de una matriz
import pandas as pd

#esta tabulate nos sirve para hacer tablas
from tabulate import tabulate

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # -> Necesario para graficar superficies 3D

#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split

# Modulo de optimizacion en scipy
from scipy import optimize

### 1.1 Cargamos los datos del dataset
cargamos los datos haciendo el uso de la libreria **Pandas** que  es una herramienta poderosa y versátil utilizada para manipulación y análisis de datos. Ofrece estructuras de datos flexibles y eficientes para trabajar con datos tabulares, como hojas de cálculo en Excel o tablas SQL.


In [527]:
#Haciendo uso de la libreria pandas para leer el dataset, delimitado por ","
dataset = pd.read_csv('credit_score.csv', delimiter=',')

#ahora tenemos un dataframe
#Imprimimos en una tabla el dataset para hacer un analisis mas claro.
dataset

,Unnamed: 0,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,1,2,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,31.944960,0,1,49.574949,118.280222,3,284.629162,2
1,2,6,34.429817,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,28.609352,267,1,49.574949,81.699521,4,331.209863,2
2,3,0,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,31.377862,268,1,49.574949,199.458074,5,223.451310,2
3,4,7,23.000000,13,19114.12,1824.843333,3.0,4.0,3.000000,4.0,...,1,809.98,24.797347,269,1,49.574949,41.420153,1,341.489231,2
4,5,5,23.000000,13,19114.12,4194.170850,3.0,4.0,3.000000,4.0,...,1,809.98,27.262259,270,1,49.574949,62.430172,6,340.479212,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79800,99991,1,29.000000,1,20002.88,1929.906667,10.0,8.0,29.000000,5.0,...,0,3571.70,37.140784,75,2,60.964772,34.662906,0,337.362988,1
79801,99993,2,25.000000,9,39628.99,3359.415833,4.0,6.0,7.000000,2.0,...,1,502.38,29.135447,376,1,58638.000000,180.733095,4,400.104466,1
79802,99997,5,25.000000,9,39628.99,3359.415833,4.0,6.0,14.546679,2.0,...,1,502.38,41.255522,380,1,35.104023,24.028477,0,516.809083,0
79803,99998,4,25.000000,9,39628.99,3359.415833,4.0,6.0,7.000000,2.0,...,1,502.38,33.638208,381,1,35.104023,251.672582,3,319.164979,1


mostrando la tabla nos damos cuenta que nuestra Y a predicir es el **Credit_Score**, para tener una mejor vision de la cantidad de clases que existe, se hizo el siguiente codigo:

donde separamos nuestra columna Y con `value_counts()` este método de pandas cuenta el número de veces que aparece cada valor único en la columna **Credit_Score** del DataFrame. Devuelve una Serie pandas donde los índices son los valores únicos de la columna **Credit_Score** y los valores son el recuento de ocurrencias de cada valor.

In [528]:
#para contar cuantas clases contiene nuestra columna y
class_counts = dataset["Credit_Score"].value_counts()

#para contar cuantas caracteristicas tiene nuestro dataset, obviamente con sin contar nuestra y, por eso lo dropeamos
feactures_counts =dataset.drop(['Credit_Score'], axis=1)
feactures_counts = feactures_counts.shape[1]

print(f"la cantidad de caracteristicas es: { feactures_counts}")
print(f'la cantidad de clases es: 3')

#mostramos la cantidad de clases tiene, y en que cantidad
class_counts

la cantidad de caracteristicas es: 24
la cantidad de clases es: 3


Credit_Score
1    42470
0    23129
2    14206
Name: count, dtype: int64

### 1.3 Analisis del dataset:

Hay 3 clases diferentes de puntajes crediticios: etiquetados como 0, 1 y 2.

* La clase 1 parece ser la más común, seguida por la clase 0 y luego la clase 2.
* El recuento de cada clase es el siguiente:
    - Clase 0: 23129 personas.
    - Clase 1: 42470 personas.
    - Clase 2: 14206 personas.
* Este análisis de la distribución de los puntajes crediticios en el conjunto de datos es útil para comprender la proporción de cada clase. Esto puede ser relevante al desarrollar un modelo de clasificación como la Regresión Logística one vs all. 

In [529]:
#almacenamos la cantidad de caracteristicas en una variable
input_layer_size  = feactures_counts;

#almacenamos la cantidad de clases en una variable
num_labels = 3;

### 1.3 Separacion de datos
Se debe tomar en cuenta que cada clase tiene su propia cantidad, por lo cual separarlos directamente en un 80% para entrenamiento y un 20% para test no seria tan efectivo, ya que puede que en el 80% hay mas datos de una clase que las otras, provocando que nuestro modelo no conozca mucho sobre esa clase, por lo cual debe separarse un 80% para entrenamiento y un 20% para pruebas de cada clase.

Para este hacemos uso de la libreria **train_test_split**

In [530]:
# hacemos uso del DataFrame llamado 'dataset' que contiene nuestros datos datos
# y es la columna que contiene las etiquetas de clase (en este caso, 'Credit_Score')

#creamos una variable temporal que contentra toda la columna de 'Credit_Score'
y_temp = dataset['Credit_Score']

# Para la clase 0

#donde y_temp es igual a 0, separamos los datos en train_class_0, test_class_0
data_class_0 = dataset[y_temp == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1

#donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
data_class_1 = dataset[y_temp == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

# Para la clase 2

#donde y_temp es igual a 2, separamos los datos en train_class_2, test_class_2
data_class_2 = dataset[y_temp == 2]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)

haciendo conteo de separacion de datos:


In [531]:
print(f"Para la clase 0 tenemos una cantidad de: { data_class_0.shape[0]} donde el 80% es: {train_class_0.shape[0]} y el 20% es: {test_class_0.shape[0]}")
print(f"Para la clase 1 tenemos una cantidad de: { data_class_1.shape[0]} donde el 80% es: {train_class_1.shape[0]} y el 20% es: {test_class_1.shape[0]}")
print(f"Para la clase 2 tenemos una cantidad de: { data_class_2.shape[0]} donde el 80% es: {train_class_2.shape[0]} y el 20% es: {test_class_2.shape[0]}")
print(f"La cantidad total de datos es: {dataset.shape[0]}")

Para la clase 0 tenemos una cantidad de: 23129 donde el 80% es: 18503 y el 20% es: 4626
Para la clase 1 tenemos una cantidad de: 42470 donde el 80% es: 33976 y el 20% es: 8494
Para la clase 2 tenemos una cantidad de: 14206 donde el 80% es: 11364 y el 20% es: 2842
La cantidad total de datos es: 79805


Separamos cada uno en sus X_train, y_train, X_test y y_test respectivos:

In [532]:
#para la parte de entrenamiento, separamos las caracteristicas de la etiqueta

#para la clase 0
X_train_class_0 = train_class_0.drop(['Credit_Score'], axis=1)
y_train_class_0 = train_class_0['Credit_Score']

#para la clase 1
X_train_class_1 = train_class_1.drop(['Credit_Score'], axis=1)
y_train_class_1 = train_class_1['Credit_Score']

#para la clase 2
X_train_class_2 = train_class_2.drop(['Credit_Score'], axis=1)
y_train_class_2 = train_class_2['Credit_Score']

#ahora para la parte de pruebas, separamos las caracteristicas de la etiqueta
#para la clase 0
X_test_class_0 = test_class_0.drop(['Credit_Score'], axis=1)
y_test_class_0 = test_class_0['Credit_Score']

#para la clase 1
X_test_class_1 = test_class_1.drop(['Credit_Score'], axis=1)
y_test_class_1 = test_class_1['Credit_Score']

#para la clase 2
X_test_class_2 = test_class_2.drop(['Credit_Score'], axis=1)
y_test_class_2 = test_class_2['Credit_Score']

ahora unimos todos en una sola matriz para X y y:
pero luego debemos mezclar los datos, haciendo uso de ``np.random.permutation(len(X))`` genera un arreglo de índices permutados aleatoriamente.
Luego, estos índices se usan para reorganizar tanto las características como las etiquetas de entrenamiento y prueba.

In [533]:
#separando los datos de entrenamiento y pruebas

#para los datos de entrenamiento
X_train = pd.concat([X_train_class_0, X_train_class_1, X_train_class_2]).values
y_train = pd.concat([y_train_class_0, y_train_class_1, y_train_class_2]).values


indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = len(y_train)

#para los datos de pruebas
X_test = pd.concat([X_test_class_0, X_test_class_1, X_test_class_2]).values
y_test = pd.concat([y_test_class_0, y_test_class_1, y_test_class_2]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
m_test = len(y_test)

Visualizamos algunos datos:

In [534]:
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>6s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]', 'X[:, 12]', 
    'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]', 'X[:, 20]', 'X[:, 21]', 'X[:, 22]', 'X[:, 23]','Y'
))
print('-' * 250)

for i in range(10):
    print('{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:8.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:8.0f}'.format(
        X_train[i, 0],
        X_train[i, 1],
        X_train[i, 2], 
        X_train[i, 3], 
        X_train[i, 4], 
        X_train[i, 5], 
        X_train[i, 6], 
        X_train[i, 7], 
        X_train[i, 8], 
        X_train[i, 9], 
        X_train[i, 10],
        X_train[i, 11],
        X_train[i, 12], 
        X_train[i, 13], 
        X_train[i, 14], 
        X_train[i, 15], 
        X_train[i, 16],
        X_train[i, 17],
        X_train[i, 18],
        X_train[i, 19], 
        X_train[i, 20], 
        X_train[i, 21], 
        X_train[i, 22], 
        X_train[i, 23],
        y_train[i]
    ))

#mostramos la cantidad de ejemplos
print(" ")
print('El 80% de ejemplos para entrenamiento son la cantidad de: {:.0f} de ejemplos'.format( len(X_train)))
print('El 20% de ejemplos para pruebas son la cantidad de: {:.0f} de ejemplos'.format(  len(X_test)))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]  X[:, 12]  X[:, 13]  X[:, 14]  X[:, 15]  X[:, 16]  X[:, 17]  X[:, 18]  X[:, 19]  X[:, 20]  X[:, 21]  X[:, 22]  X[:, 23]     Y
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
     92452         7        28         9     35176      2815       4         6         8         4      5249        32        20      3242         1         2       235        26       191         2       105        80         6       347       1
     12838         4        29         9     33900      4194       6         5        12         3      5107        22        20      1681      1166         2       593        32       230         0        57        57         1       414       2
     538

### 1.2 Normalización de caracteristicas

Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor en una escala de valores similares, esto con el fin de que el descenso por el gradiente pueda converger mas rapidamente. Se aplica la normalizacion esto debido a que los datos de las X estan a diferentes escalas.

Hacemos el uso de la siguiente funcion para normalizar los datos de las columnas X:

In [535]:
def  featureNormalize(X):
    X_norm = X.copy()

    #creamos un array de ceros con una longitud igual al número de columnas en el array X. La variable mu y sigma se inicializa como este array de ceros.
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #Creamos el promedio de cada columna de X
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    
    sigma[sigma == 0] = 1
    
    #normalizamos los datos con la siguiente formula
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

Normalizando las caracteristicas:

In [536]:
X_norm, mu, sigma= featureNormalize(X_train)

# imprimir todos las X_norm de datos solo 10
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>6s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]', 'X[:, 12]', 
    'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]', 'X[:, 20]', 'X[:, 21]', 'X[:, 22]', 'X[:, 23]','Y'
))
print('-' * 250)

for i in range(10):
    print('{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:10.2f}{:8.0f}'.format(
        X_norm[i, 0],
        X_norm[i, 1],
        X_norm[i, 2], 
        X_norm[i, 3], 
        X_norm[i, 4], 
        X_norm[i, 5], 
        X_norm[i, 6], 
        X_norm[i, 7], 
        X_norm[i, 8], 
        X_norm[i, 9], 
        X_norm[i, 10],
        X_norm[i, 11],
        X_norm[i, 12], 
        X_norm[i, 13], 
        X_norm[i, 14], 
        X_norm[i, 15], 
        X_norm[i, 16],
        X_norm[i, 17],
        X_norm[i, 18],
        X_norm[i, 19], 
        X_norm[i, 20], 
        X_norm[i, 21], 
        X_norm[i, 22], 
        X_norm[i, 23],
        y_train[i]
    ))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]  X[:, 12]  X[:, 13]  X[:, 14]  X[:, 15]  X[:, 16]  X[:, 17]  X[:, 18]  X[:, 19]  X[:, 20]  X[:, 21]  X[:, 22]  X[:, 23]     Y
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
      1.47      1.53     -0.66      0.33     -0.10     -0.47     -0.47     -0.00     -0.75      0.18      0.88      0.73     -0.05      0.59     -0.14      0.97     -1.03     -1.17     -0.09      0.86     -0.16     -0.28      1.52     -0.26       1
     -1.29      0.22     -0.56      0.33     -0.10     -0.00      0.20     -0.21     -0.29     -0.22      0.81      0.06     -0.05     -0.68      6.00      0.97     -0.72      0.03      0.25     -2.02     -0.16     -0.29     -1.01      0.05       2
    

### 1.3 Creacion de la funcion Sigmoide
También conocida como la función logística, es una función matemática que toma cualquier número real como entrada y devuelve un valor en el rango de 0 a 1. Donde nuestra **Z** es nuestra hipotesis.

Creando la funcion:

In [537]:
def sigmoid(z):
    z = np.array(z)

    g = np.zeros(z.shape)

    g = 1 / (1 + np.exp(-z))

    return g

### 1.4 Vectorización de regresión logística


#### 1.3.1 Vectorización de la funcion de costo

In [538]:
def lrCostFunction(theta, X, y, lambda_):
    #cremos una variable m que contiene la longitud de y
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    #inicializamos J y grad
    J = 0
    grad = np.zeros(theta.shape)

    #calculamos h haciendo uso de la funcion sigmoid
    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

## 1.4 Clasificacion One-vs-All

In [539]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [540]:
lambda_ = 0.005
all_theta = oneVsAll(X_norm, y_train, num_labels, lambda_)
print(all_theta.shape)

(3, 25)


In [541]:
for i in range(all_theta.shape[1]):
    print("Thetha",i," para la clase 0:", all_theta[0,i]," ; ", "Thetha",i," para la clase 1:", all_theta[1,i]," ; ", "Thetha",i," para la clase 2:", all_theta[2,i])

Thetha 0  para la clase 0: -1.0816784883074766  ;  Thetha 0  para la clase 1: 0.14286616335560176  ;  Thetha 0  para la clase 2: -2.371160789101217
Thetha 1  para la clase 0: -0.020633914276253634  ;  Thetha 1  para la clase 1: 0.01702527977832005  ;  Thetha 1  para la clase 2: -0.009365204089549517
Thetha 2  para la clase 0: 0.0014147414700806155  ;  Thetha 2  para la clase 1: 0.016848208875444106  ;  Thetha 2  para la clase 2: -0.03735205756286588
Thetha 3  para la clase 0: -0.06068367988408213  ;  Thetha 3  para la clase 1: 0.0071295601585379415  ;  Thetha 3  para la clase 2: 0.04985254637727971
Thetha 4  para la clase 0: 0.006882137377106501  ;  Thetha 4  para la clase 1: -0.0009403914407025093  ;  Thetha 4  para la clase 2: -0.01493187777895309
Thetha 5  para la clase 0: -0.024970237891034145  ;  Thetha 5  para la clase 1: 0.021184829003999563  ;  Thetha 5  para la clase 2: -0.0010891823117569145
Thetha 6  para la clase 0: -0.09469458844833442  ;  Thetha 6  para la clase 1: -0.004

#### 1.4.1 Prediccion One-vs-all

In [542]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [543]:
# print(X_test.shape)
pred = predictOneVsAll(all_theta, X_norm)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_train) * 100))



Precision del conjuto de entrenamiento: 62.02%


In [544]:

X_test = (X_test - mu) / sigma
X_test = np.concatenate([np.ones((len(X_test), 1)), X_test], axis=1)

print(X_test.shape)

p = np.argmax(sigmoid(X_test.dot(all_theta.T)), axis = 1)

for i in range(30):
    print(f"Prediccion: {p[i]}, Real: {y_test[i]}")

(15962, 25)
Prediccion: 1, Real: 0
Prediccion: 2, Real: 2
Prediccion: 0, Real: 0
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 0, Real: 0
Prediccion: 1, Real: 1
Prediccion: 1, Real: 0
Prediccion: 1, Real: 1
Prediccion: 0, Real: 0
Prediccion: 2, Real: 2
Prediccion: 1, Real: 1
Prediccion: 1, Real: 0
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 2, Real: 2
Prediccion: 1, Real: 1
Prediccion: 0, Real: 0
Prediccion: 1, Real: 2
Prediccion: 1, Real: 2
Prediccion: 0, Real: 1
Prediccion: 1, Real: 1
Prediccion: 1, Real: 0
Prediccion: 1, Real: 1
Prediccion: 1, Real: 2
Prediccion: 2, Real: 0
Prediccion: 2, Real: 2
Prediccion: 2, Real: 1
Prediccion: 1, Real: 1
